# PROJET « dur » : prédiction de match de ligue1

Le but du projet est de prédire le résultat (victoire, nul, défaite) des matchs de ligue 1 sur la saison en cours (2023-2024).

In [94]:
#importation des librairie
import numpy as np
import pandas as pd 
import sklearn as sk 
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

## Importation des données

In [95]:
clubs = pd.read_csv("data/clubs_fr.csv", sep=",") # Une liste des clubs français avec quelques stats sur la constitution (récente) de l’équipe
game_events = pd.read_csv("data/game_events.csv", sep=",") # Un ensemble d’actions pour chaque joueur pendant chaque match
game_lineups = pd.read_csv("data/game_lineups.csv", sep=",", low_memory=False) # La constitution des équipes pour chaque match
match2023 = pd.read_csv("data/match_2023.csv", sep=",") # matchs a predir
matchs = pd.read_csv("data/matchs_2013_2022.csv", sep=",") #matchs de 2013 a 2022
player_appearance = pd.read_csv("data/player_appearance.csv", sep=",") # Un ensemble d’info simple sur chaque joueur pendant chaque match
pre_season = pd.read_csv("data/player_valuation_before_season.csv", sep=",") # Pour chaque joueur, sa valeur sur le marché, à une certaine date

## Nettoyage

In [96]:
clubs.dropna(axis=0, how="all")
clubs.dropna(axis=1, how="all")

game_events.dropna(axis=0, how="all")
game_events.dropna(axis=1, how="all")

game_lineups.dropna(axis=0, how="all")
game_lineups.dropna(axis=1, how="all")

matchs.dropna(axis=0, how="all")
matchs.dropna(axis=1, how="all")

player_appearance.dropna(axis=0, how="all")
player_appearance.dropna(axis=1, how="all")

pre_season.dropna(axis=0, how="all")
pre_season.dropna(axis=1, how="all")

,player_id,date,market_value_in_eur,current_club_id,player_club_domestic_competition_id
0,773,2004-10-04,3500000,14171,FR1
1,1327,2004-10-04,4000000,1159,FR1
2,1423,2004-10-04,1000000,855,FR1
3,1572,2004-10-04,1000000,162,FR1
4,1613,2004-10-04,200000,855,FR1
...,...,...,...,...,...
30210,478872,2023-07-27,500000,618,FR1
30211,550862,2023-07-27,450000,1420,FR1
30212,363717,2023-07-28,150000,1162,FR1
30213,396131,2023-07-28,150000,1421,FR1


## Exploration des données

Tout d'abord, nous allons voir quels sont les données dont nous disposons 

In [97]:
print("clubs : ",clubs.columns)
print("game_events : ",game_events.columns)
print("game_lineups : ",game_lineups.columns)
print("matchs : ",matchs.columns)
print("player_appearance : ",player_appearance.columns)
print("pre_season : ",pre_season.columns)

clubs :  Index(['club_id', 'club_code', 'name', 'domestic_competition_id', 'squad_size',
       'average_age', 'foreigners_number', 'foreigners_percentage',
       'national_team_players', 'stadium_name', 'stadium_seats',
       'net_transfer_record', 'coach_name'],
      dtype='object')
game_events :  Index(['Unnamed: 0', 'game_event_id', 'date', 'game_id', 'minute', 'type',
       'club_id', 'player_id', 'description', 'player_in_id',
       'player_assist_id'],
      dtype='object')
game_lineups :  Index(['Unnamed: 0', 'game_lineups_id', 'date', 'game_id', 'player_id',
       'club_id', 'player_name', 'type', 'position', 'number', 'team_captain'],
      dtype='object')
matchs :  Index(['Unnamed: 0', 'game_id', 'season', 'round', 'date', 'home_club_id',
       'away_club_id', 'home_club_goals', 'away_club_goals',
       'home_club_position', 'away_club_position', 'home_club_manager_name',
       'away_club_manager_name', 'stadium', 'attendance', 'referee',
       'home_club_formation

### Fonctions d'aggregation de données 

Dans player_appearance on ne garde pas les elements suivants: appearance_id, game_id, competition_id et on va ajouter le nombre de minutes de jeu moyen

In [98]:
def aggregate_player_stats_by_year(player_appearance):

    # Conversion de la colonne 'date' en datetime
    player_appearance['date'] = pd.to_datetime(player_appearance['date'])
    
    # Extraction de l'année de la colonne 'date'
    player_appearance['year'] = player_appearance['date'].dt.year
    
    # Renommer la colonne 'player_club_id' en 'club_id'
    player_appearance = player_appearance.rename(columns={'player_club_id': 'club_id'})
    
    # Agrégation des statistiques des joueurs par joueur, année et club
    stats = player_appearance.groupby(['player_id', 'year', 'club_id']).agg({
        'yellow_cards': 'sum',
        'red_cards': 'sum',
        'goals': 'sum',
        'assists': 'sum',
        'minutes_played': ['sum', 'mean'],  # Ajout de la somme et de la moyenne des minutes jouées
    }).reset_index()

    # Renommer les colonnes agrégées pour la clarté
    stats.columns = ['player_id', 'year', 'club_id', 'yellow_cards', 'red_cards', 'goals', 'assists', 'total_minutes_played', 'avg_minutes_played']
    
    return stats

aggregated_player_appearance = aggregate_player_stats_by_year(player_appearance).sort_values(by='year')
aggregated_player_appearance

,player_id,year,club_id,yellow_cards,red_cards,goals,assists,total_minutes_played,avg_minutes_played
3212,126542,2012,1095,1,0,0,0,783,60.230769
810,24700,2012,1158,1,0,1,0,495,82.500000
3203,126530,2012,1158,3,0,0,1,450,56.250000
814,25113,2012,3911,2,0,0,1,979,65.266667
817,25508,2012,583,2,0,0,0,1530,90.000000
...,...,...,...,...,...,...,...,...,...
4637,202278,2023,273,0,0,1,2,574,41.000000
5398,263951,2023,290,0,0,0,1,373,33.909091
7620,593482,2023,244,0,0,1,0,115,16.428571
7610,592979,2023,1420,3,0,2,2,1824,82.909091


Dans pre_season on retire les elements suivants : player_club_domestic_competition_id

In [99]:
def aggregate_player_pre_season_stats_by_year(df):

    #on veux seulement l'année de la saison
    df['date'] = pd.to_datetime(df['date'])
    df['year'] = pre_season['date'].dt.year
    df = df.rename(columns={'current_club_id': 'club_id'})
    df.drop(columns=['player_club_domestic_competition_id'])
    df.sort_values(by="year")

    
    return df

pre_season = aggregate_player_pre_season_stats_by_year(pre_season)
pre_season

,player_id,date,market_value_in_eur,club_id,player_club_domestic_competition_id,year
0,773,2004-10-04,3500000,14171,FR1,2004
1,1327,2004-10-04,4000000,1159,FR1,2004
2,1423,2004-10-04,1000000,855,FR1,2004
3,1572,2004-10-04,1000000,162,FR1,2004
4,1613,2004-10-04,200000,855,FR1,2004
...,...,...,...,...,...,...
30210,478872,2023-07-27,500000,618,FR1,2023
30211,550862,2023-07-27,450000,1420,FR1,2023
30212,363717,2023-07-28,150000,1162,FR1,2023
30213,396131,2023-07-28,150000,1421,FR1,2023


Pour matchs, on a rajouter les valeurs home wins/lose, away win/lose, home/away goals

In [100]:
def aggregate_all_match_stats(df):
    # Convertir la colonne de date en format datetime
    df['date'] = pd.to_datetime(df['date'])
    
    # Extraire l'année à partir de la colonne de date
    df['year'] = df['date'].dt.year
    
    # Séparer les données pour les clubs à domicile et à l'extérieur
    home_stats = df[['year', 'home_club_id', 'home_club_goals', 'away_club_goals', 'results']].copy()
    away_stats = df[['year', 'away_club_id', 'away_club_goals', 'home_club_goals', 'results']].copy()
    
    # Renommer les colonnes pour faciliter l'agrégation
    home_stats = home_stats.rename(columns={
        'home_club_id': 'club_id',
        'home_club_goals': 'goals_for',
        'away_club_goals': 'goals_against',
        'results': 'result'
    })
    away_stats = away_stats.rename(columns={
        'away_club_id': 'club_id',
        'away_club_goals': 'goals_for',
        'home_club_goals': 'goals_against',
        'results': 'result'
    })
    
    # Ajouter une colonne pour indiquer si le club était à domicile ou à l'extérieur
    home_stats['home'] = True
    away_stats['home'] = False
    
    # Fusionner les deux DataFrames
    all_stats = pd.concat([home_stats, away_stats])
    
    # Calculer les points pour chaque match
    def calculate_points(row):
        if row['home'] and row['result'] == 1:
            return 3
        elif not row['home'] and row['result'] == -1:
            return 3
        elif row['result'] == 0:
            return 1
        else:
            return 0

    all_stats['points'] = all_stats.apply(calculate_points, axis=1)
    
    # Calculer les indicateurs de victoire et défaite
    all_stats['home_victory'] = ((all_stats['home'] == True) & (all_stats['result'] == 1)).astype(int)
    all_stats['away_victory'] = ((all_stats['home'] == False) & (all_stats['result'] == 1)).astype(int)
    all_stats['home_lose'] = ((all_stats['home'] == True) & (all_stats['result'] == -1)).astype(int)
    all_stats['away_lose'] = ((all_stats['home'] == False) & (all_stats['result'] == -1)).astype(int)
    
    # Grouper les données par club et par année
    grouped_stats = all_stats.groupby(['club_id', 'year']).agg({
        'goals_for': 'sum',
        'goals_against': 'sum',
        'points': 'sum',
        'result': 'count',  # Nombre de matchs joués
        'home_victory': 'sum',
        'away_victory': 'sum',
        'home_lose': 'sum',
        'away_lose': 'sum'
    }).reset_index()
    
    # Renommer la colonne 'result' en 'matches_played'
    grouped_stats = grouped_stats.rename(columns={'result': 'matches_played'})
    grouped_stats.sort_values(by="year")
    
    return grouped_stats

# Exemple d'utilisation
# Supposons que matchs soit déjà chargé comme DataFrame
# matchs = pd.read_csv('path_to_your_csv_file.csv')
match_stats_by_year = aggregate_all_match_stats(matchs)
match_stats_by_year

,club_id,year,goals_for,goals_against,points,matches_played,home_victory,away_victory,home_lose,away_lose
0,40,2012,21,14,29,19,3,2,0,3
1,40,2013,46,41,57,38,11,5,6,4
2,40,2014,48,49,53,38,10,10,3,4
3,40,2015,44,45,55,38,11,6,3,2
4,40,2016,47,55,52,38,7,7,5,6
...,...,...,...,...,...,...,...,...,...,...
261,3911,2023,26,22,31,22,6,4,3,2
262,14171,2012,22,32,19,19,2,7,3,3
263,14171,2013,43,52,41,37,7,10,6,3
264,14171,2014,40,52,43,38,8,12,7,4


In [101]:
aggregated_player_appearance.columns
game_lineups.columns

Index(['Unnamed: 0', 'game_lineups_id', 'date', 'game_id', 'player_id',
       'club_id', 'player_name', 'type', 'position', 'number', 'team_captain'],
      dtype='object')

Pour game_events, on ne va rien modifier.

Ensuite on va utiliser game_lineups pour ajouter la position/le numero le plus frequent ainsi que le nombre d'arret du joueur

In [104]:
def add_player_position(game_lineups_df, aggregated_player_appearance_df):
    # Grouper le DataFrame game_lineups par player_id et obtenir la position la plus fréquente
    most_frequent_position = game_lineups_df.groupby('player_id')['position'].agg(lambda x: x.mode()[0]).reset_index()
    
    # Fusionner les positions les plus fréquentes avec aggregated_player_appearance_df
    aggregated_player_appearance_df = pd.merge(aggregated_player_appearance_df, most_frequent_position, on='player_id', how='left')
    most_frequent_number = game_lineups_df.groupby('player_id')['number'].agg(lambda x: x.mode()[0]).reset_index()
    
    # Fusionner les numéros de joueur les plus fréquents avec aggregated_player_appearance_df
    aggregated_player_appearance_df = pd.merge(aggregated_player_appearance_df, most_frequent_number, on='player_id', how='left')
    
    return aggregated_player_appearance_df

# Exemple d'utilisation
# Supposons que game_lineups et aggregated_player_appearance soient déjà chargés comme DataFrames
# game_lineups = pd.read_csv('path_to_your_csv_file.csv')
# aggregated_player_appearance = pd.read_csv('path_to_your_csv_file.csv')
aggregated_player_appearance = add_player_position(game_lineups, aggregated_player_appearance)
aggregated_player_appearance[aggregated_player_appearance["player_id"]==25508]


,player_id,year,club_id,yellow_cards,red_cards,goals,assists,total_minutes_played,avg_minutes_played,position_x,number_x,position_y,number_y
4,25508,2012,583,2,0,0,0,1530,90.000000,Goalkeeper,30,Goalkeeper,30
1129,25508,2013,583,0,0,0,0,3107,88.771429,Goalkeeper,30,Goalkeeper,30
1807,25508,2014,583,1,0,0,0,3330,90.000000,Goalkeeper,30,Goalkeeper,30
2077,25508,2015,583,1,0,0,0,1383,86.437500,Goalkeeper,30,Goalkeeper,30
2657,25508,2016,583,0,0,0,0,135,67.500000,Goalkeeper,30,Goalkeeper,30
